In [1]:
import hydra
from hydra.utils import instantiate


In [2]:
def load_config(config_name="evaluate", overrides=None):
    # 初始化 Hydra 上下文
    with hydra.initialize(config_path="config", version_base=None):
        # 加载配置，允许覆盖参数
        cfg = hydra.compose(config_name=config_name, overrides=overrides or [])
    return cfg

In [20]:
cfg = load_config(overrides=[
                "task=libero_long",
                "algo=act",
                "exp_name=pretrain_act_libero_90",
                "variant_name=block_16",
                "stage=2",
                "algo.skill_block_size=16",
                "training.use_tqdm=true",
                "seed=0",
                "rollout.n_video=1",
                'rollout.rollouts_per_env=5',
                "checkpoint_path=/home/yeyifan/workplace/embodied/experiments/libero/LIBERO_90/act/pretrain_act_libero_90/block_16/0/run_002/multitask_model_epoch_0100.pth"
            ],)
from omegaconf import OmegaConf
OmegaConf.register_new_resolver("eval", eval, replace=True)
OmegaConf.resolve(cfg)

In [21]:
cfg.rollout

{'enabled': True, 'interval': 25, 'rollouts_per_env': 5, 'max_episode_length': 600, 'num_parallel_envs': 5, 'n_video': 1}

In [22]:
import quest.utils.utils as utils
import torch 

device = cfg.device
seed = cfg.seed
torch.manual_seed(seed)
train_cfg = cfg.training
checkpoint_path = utils.get_latest_checkpoint(cfg.checkpoint_path)
state_dict = utils.load_state(checkpoint_path)

model = instantiate(cfg.algo.policy,shape_meta=cfg.task.shape_meta)
model.to(device)
model.eval()
model.load_state_dict(state_dict['model'])

<All keys matched successfully>

In [23]:
env_runner = instantiate(cfg.task.env_runner)

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [24]:
env_runner.env_names

['LIVING_ROOM_SCENE2_put_both_the_alphabet_soup_and_the_tomato_sauce_in_the_basket',
 'LIVING_ROOM_SCENE2_put_both_the_cream_cheese_box_and_the_butter_in_the_basket',
 'KITCHEN_SCENE3_turn_on_the_stove_and_put_the_moka_pot_on_it',
 'KITCHEN_SCENE4_put_the_black_bowl_in_the_bottom_drawer_of_the_cabinet_and_close_it',
 'LIVING_ROOM_SCENE5_put_the_white_mug_on_the_left_plate_and_put_the_yellow_and_white_mug_on_the_right_plate',
 'STUDY_SCENE1_pick_up_the_book_and_place_it_in_the_back_compartment_of_the_caddy',
 'LIVING_ROOM_SCENE6_put_the_white_mug_on_the_plate_and_put_the_chocolate_pudding_to_the_right_of_the_plate',
 'LIVING_ROOM_SCENE1_put_both_the_alphabet_soup_and_the_cream_cheese_box_in_the_basket',
 'KITCHEN_SCENE8_put_both_moka_pots_on_the_stove',
 'KITCHEN_SCENE6_put_the_yellow_and_white_mug_in_the_microwave_and_close_it']

In [32]:
import numpy as np
from moviepy import ImageSequenceClip
import os
def save_rollout(rollouts,prompt):
    flag = os.path.exists(f"rollouts/act/{prompt}")
    if not flag:
        os.makedirs(f"rollouts/act/{prompt}")
    for i,(_,_,episode) in enumerate(rollouts):
        video_hwc = np.array(episode['render'])
        video_chw = video_hwc.transpose((0, 3, 1, 2))
        clip = ImageSequenceClip(list(video_chw.transpose(0, 2, 3, 1)), fps=24)
        if flag:
            clip.write_videofile(f"rollouts/act/{prompt}/new_video_{i}.mp4")
        else:            
            clip.write_videofile(f"rollouts/act/{prompt}/video_{i}.mp4")


In [33]:
prompt = 'LIVING_ROOM_SCENE2_move_to_the_basket'
rollouts = env_runner.run_policy_in_env(env_runner.env_names[0],model,render=True,prompt=prompt)
save_rollout(rollouts,prompt)

/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/wandb/analytics/sentry.py:90: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  self.hub = sentry_sdk.Hub(client)
/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/wandb/analytics/sentry.py:90: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  self.hub = sentry_sdk.Hub(client)
/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/wandb/analytics/sentry.py:90: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will 

MoviePy - Building video rollouts/act/LIVING_ROOM_SCENE2_move_to_the_basket/video_0.mp4.
MoviePy - Writing video rollouts/act/LIVING_ROOM_SCENE2_move_to_the_basket/video_0.mp4



frame_index:   0%|          | 3/601 [02:08<7:05:41, 42.71s/it, now=None]

MoviePy - Done !
MoviePy - video ready rollouts/act/LIVING_ROOM_SCENE2_move_to_the_basket/video_0.mp4
MoviePy - Building video rollouts/act/LIVING_ROOM_SCENE2_move_to_the_basket/video_1.mp4.
MoviePy - Writing video rollouts/act/LIVING_ROOM_SCENE2_move_to_the_basket/video_1.mp4



frame_index:   0%|          | 3/601 [02:08<7:06:16, 42.77s/it, now=None]

...

MoviePy - Done !
MoviePy - video ready rollouts/act/LIVING_ROOM_SCENE2_move_to_the_basket/video_3.mp4
MoviePy - Building video rollouts/act/LIVING_ROOM_SCENE2_move_to_the_basket/video_4.mp4.
MoviePy - Writing video rollouts/act/LIVING_ROOM_SCENE2_move_to_the_basket/video_4.mp4



frame_index:   0%|          | 3/601 [02:09<7:08:44, 43.02s/it, now=None]Exception ignored in: <function MjRenderContext.__del__ at 0x7fe11ddf9f30>
Traceback (most recent call last):
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/robosuite/utils/binding_utils.py", line 199, in __del__
    self.gl_ctx.free()
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/robosuite/renderers/context/egl_context.py", line 149, in free
    EGL.eglMakeCurrent(EGL_DISPLAY, EGL.EGL_NO_SURFACE, EGL.EGL_NO_SURFACE, EGL.EGL_NO_CONTEXT)
  File "src/errorchecker.pyx", line 59, in OpenGL_accelerate.errorchecker._ErrorChecker.glCheckError
OpenGL.raw.EGL._errors.EGLError: EGLError(
	err = EGL_NOT_INITIALIZED,
	baseOperation = eglMakeCurrent,
	cArguments = (
	),
	result = 0
)
Exception ignored in: <function EGLGLContext.__del__ at 0x7fe11ddf9d80>
Traceback (most recent call last):
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10

MoviePy - Done !
MoviePy - video ready rollouts/act/LIVING_ROOM_SCENE2_move_to_the_basket/video_4.mp4


Exception ignored in: <function MjRenderContext.__del__ at 0x7f63fd5edf30>
Traceback (most recent call last):
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/robosuite/utils/binding_utils.py", line 199, in __del__
    self.gl_ctx.free()
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/robosuite/renderers/context/egl_context.py", line 149, in free
    EGL.eglMakeCurrent(EGL_DISPLAY, EGL.EGL_NO_SURFACE, EGL.EGL_NO_SURFACE, EGL.EGL_NO_CONTEXT)
  File "src/errorchecker.pyx", line 59, in OpenGL_accelerate.errorchecker._ErrorChecker.glCheckError
OpenGL.raw.EGL._errors.EGLError: EGLError(
	err = EGL_NOT_INITIALIZED,
	baseOperation = eglMakeCurrent,
	cArguments = (
	),
	result = 0
)
Exception ignored in: <function EGLGLContext.__del__ at 0x7f63fd5edd80>
Traceback (most recent call last):
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/robosuite/renderers/context/egl_context.py", line 155, in

Exception ignored in: <function MjRenderContext.__del__ at 0x7ff916ec1f30>
Traceback (most recent call last):
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/robosuite/utils/binding_utils.py", line 199, in __del__
    self.gl_ctx.free()
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/robosuite/renderers/context/egl_context.py", line 149, in free
    EGL.eglMakeCurrent(EGL_DISPLAY, EGL.EGL_NO_SURFACE, EGL.EGL_NO_SURFACE, EGL.EGL_NO_CONTEXT)
  File "src/errorchecker.pyx", line 59, in OpenGL_accelerate.errorchecker._ErrorChecker.glCheckError
OpenGL.raw.EGL._errors.EGLError: Exception ignored in: <function MjRenderContext.__del__ at 0x7f7604001f30>
Traceback (most recent call last):
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packages/robosuite/utils/binding_utils.py", line 199, in __del__
    self.gl_ctx.free()
  File "/home/yeyifan/soft_ware/miniconda3/envs/quest/lib/python3.10/site-packag